In questo notebook verrà introdotto il concetto di Deep Learning collegato ai task di computer vision. Più precisamente in questo notebook verranno introdotte le **Convolutional Neural Network (CNN)**, anche conosciute come convnets, **applicate a task di image classification.**

Partiamo direttamente con un approccio pratico, considerando come primo esempio il dataset MNIST, dove viene richiesto di classificare un'etichetta (da 0 a 9) ad un immagine in base alla cifra presente nell'immagine stessa.

E' possibile caricare il dataset MNIST tramite la libreria keras:

In totale si hanno 70.000 immagini 28x28 con 1 singolo canale (bianco e nero - scala di grigio), 60.000 di training e 10.000 per il testing.

In [ ]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Partiamo con il creare una semplice CNN. Nella maggior parte dei casi, una     **CNN classica e semplice è formata da una serie di Convulational e Max Pooling Layer**.

Il modello verrà creato utilizzando la** Functional API**, quindi, andremo a definire i singoli layer e li **collegheremo tra di loro per stabilire il forward pass della rete.**

Definiamo l'input del modello con la shape delle immagini che riceverà in input (28x28x1) e successivamente definiamo in **maniera alternata** dei **layer di Convulazione e di Max Pooling.**

Infine si aggiunge un **layer Flatten per far si che l'ultimo layer (dense layer)riceva in input un tensore 1D (vettore di elementi)**

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28,28,1))
x = layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu')(inputs)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)
x = layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu')(x)
x = layers.Flatten()(x)
outputs = layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)

Una volta creato il modello mostriamo la sua architettura per capire la shape dei dati di input e di output di ognuno di esso:

In questo caso,  **i layer Convoluzionale e quello di pooling**, **lavorano con tensori di rank 3 (height, width, channels)**, rispetto ai **Dense**, che **lavorano su tensori di rank 2.** Questo perchè, con le **immagini**, **si aggiunge un nuovo rank** che indica il numero di **canali di immagini** (in questo caso 1 solo).

Com'è possibile osservare dal summary, **l'altezza (height) e la larghezza (width) delle immagini tende a diminuire man mano che si va in profondità, mentre il numero dei canali aumenta** (il numero dei canali dei layer è controllato dal primo parametro Filter dei convolutional layer). Quindi più si va avanti nella rete, più le immagini vengono rimpicciolite.

Dopo l'ultimo layer Conv2D, si ha un outpur shape di (3,3,128) - **quindi una feature map 3x3 composta da 128 canali (tensore 3D)**. Il **prossimo step** è quello di dare la feature map in input a un **Dense Layer che si occupa della classificazione.**

Ma, come detto prima, i **Dense Layer lavorano** su dei tensori di dimensione minore, cioè con i vettori, che hanno** rank 1D**, mentre **l'output delll'ultimo Conv2D Layer è un tensore di rank 3D.**

**Per colmare questo gap tra i due layer**, si **aggiunge un layer intermedio** - **Flatten Layer che "appiattisce" il tensore da rank 3D a 1D** in modo da avere un vettore che rappresenta feature map.

Infine si ha il **Dense Layer con 10 unità** (in quanto si hanno 10 classi) che restituirà una distribuzione composta da 10 probabilità (quindi si una softmax come funzione di attivazione) che verrà utilizzata per classificare le immagini date in input alla rete.

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 104,202 (407.04 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

**Creato il modello, passiamo al suo allenamento.**

Avendo un problema di **classificazione multiclasse di interi** utilizzeremo sparse_categorical_crossentropy, mentre, l'ottimizzatore e le metriche non variano.

In [ ]:
from matplotlib import pyplot as plt

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 56s 57ms/step - accuracy: 0.8875 - loss: 0.3613
Epoch 2/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 79s 55ms/step - accuracy: 0.9847 - loss: 0.0500
Epoch 3/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 54ms/step - accuracy: 0.9908 - loss: 0.0302
Epoch 4/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 51s 54ms/step - accuracy: 0.9934 - loss: 0.0217
Epoch 5/5
938/938 ━━━━━━━━━━━━━━━━━━━━ 82s 54ms/step - accuracy: 0.9947 - loss: 0.0162


Una volta allenato, **testiamo il modello per capire come si comporta su dei dati mai visti.**

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy:.3f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - accuracy: 0.9888 - loss: 0.0320
Test accuracy: 0.992


# **Approfondimenti**

Scendiamo ancora più nel dettaglio per introdurre altri argomenti. Per esempio, **come mai la dimensione delle immagini, layer dopo layer, diminuiscono?**, Infatti, riprendendo la summary del modello e considerando la shape dei vari layer, man mano che si scende verso i layer finali, la dimensione delle immagini diminuisce.

L'altezza e la larghezza delle immagini di output può differire rispetto a quelle di input per due motivi principali:


*   Effetti dei bordi
*   Utilizzo dello strides

Quando si utilizza un kernel (es: 3x3 o 5x5) **non sempre è possibile centrarlo in tutte le celle delle immagini**, dipende sia dalle dimensione del kernel, che dell'immagine stessa. Per esempio, in questo esercizio, le immagini di input avevano dimensione 28x28, ma una volta date in input al primo layer convulazionale, sono diventate 26x26, sono state rimosse quindi 2 celle lungo ogni dimensione, questo perchè non era possibile centrare il filtro. \\
Se si vuole evitare si ridimensionare l'immagine dopo un operazione di convulazione, e quindi, fare in modo che le immagini di output abbiano la stessa dimensione di input, **bisogna aggiungere del padding alle immagini di input**. Il padding è una tecnica che consiste nell'aggiungere un certo numero di righe e colonne alle immagini di input (feature map di input) in modo da poter centrare il kernel in ogni zona, evitando così problemi con i bordi.

L'altro fattore che influenza la dimensione della feature map di output è la **grandezza del passo (stride)**. Quando si applica il kernel dell'operazione di convulazione, nella maggior parte dei casi, il kernel si trasla sempre di 1 (quindi il kernel si sposta di 1 cella in una direzione). Ma la grandezza del passo può essere modificata, per esempio considerando 2. **Aumentare la grandezza del passo ha la conseguenza che il kernel non verrà applicata a tutte le zone delle immagini, facendo cosi diminuire la dimensione della feature map di output**. Per esempio utilizzando uno **stride di 2, la grandezza della feature map verrà diminuita per un fattore di 2**, non di 1. Di default lo stride è sempre 1, ma può essere modificato tramite il parametro stride del layer Conv2D.



In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 208,406 (814.09 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 104,204 (407.05 KB)

# **Ulteriore approfondimento**
Nel summary del modello, la **dimensione delle immagini di output** (feature map) diminuiscono, anzi, **vengono dimezzate ogni volta che si passa per il layer MaxPooling2D** (Layer di max pooling). Per esempio: considerando il primo layer di Max Pooling, esso riceve in input una feature map 26x26, che dimezza facendola diventare 13x13.

**Questo è il compito del layer di Max Pooling: diminuire "aggressivamente" le dimensione della feature map.**

Questo layer consiste nell'applicare un kernel (solitamente 2x2) sulla feature map di output e prendere il valore massimo di ogni canale/cella. Quindi nella feature map di output ci saranno solo i **valori massimi** ottenuti dopo l'applicazione del kernel. **Ed è per questo che le dimensioni diminuiscono, in quanto si considera solo il valore massimo, escludendo gli altri.**

Questo layer è concettualmente molto simile a quello di convulazione, **la differenza** è che quest'ultima effettua delle **trasformazioni lineare** per ottenere la feature map di output, mentre il layer di **max pooling effettua l'operazione di massimo sui tensori.**

Un'altra grande differenza tra questi due layer è che:


*   **Max Pooling:** solitamente si utilizza un kernel 2x2 e uno stride di 2 in modo da diminuire la dimensione della feature map per un fattore di 2.
*   **Convulazione:** solitamente si utilizza un kernel 3x3 e non utilizza stride (stride = 1).



In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 3, 3, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │        11,530 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 208,406 (814.09 KB)

 Trainable params: 104,202 (407.04 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 104,204 (407.05 KB)